In [40]:
import torch
import torch.nn as nn
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torchshow as ts
import torchmetrics

In [81]:
training_images_letters = torchvision.datasets.EMNIST( root='data' , split = 'letters', transform = ToTensor(), download = True, train = True, target_transform=lambda y: y - 1)
training_dataloader_letters = DataLoader(training_images_letters, batch_size=1000, shuffle= True)

testing_images_letters = torchvision.datasets.EMNIST( root='data' , split = 'letters', transform = ToTensor(), download = True, train = False, target_transform=lambda y: y - 1)
testing_dataloader_letters = DataLoader(testing_images_letters, batch_size=1000)

print(f"Training set size: {len(training_images_letters)}")
print(f"Testing set size: {len(testing_images_letters)}")  

Training set size: 124800
Testing set size: 20800


In [82]:
accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=26) 

net = nn.Sequential(
    nn.Conv2d(1, 20, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2), 
    nn.Conv2d(20, 10, kernel_size=3),
    nn.ReLU(),
    nn.Conv2d(10, 10, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2), 
    nn.Flatten(),
    nn.Linear(250,26), 
)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01, maximize=False)

In [83]:
# Loop'er over 10 epoker, og udregner loss'et og accuracy for hvert.
def training_loop(training_dataloader_letters, optimizer, loss_fn):
    total_loss = 0
    accuracy.reset()
    size = len(training_dataloader_letters)
    for images,labels in training_dataloader_letters:
        optimizer.zero_grad()
        output = net(images)
        loss = loss_fn(output, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        accuracy.update(output, labels)
    avg_loss = total_loss / size
    print(f"Avg Training Accuracy: {accuracy.compute() * 100:.2f}%")
    print(f"Avg Training Loss: {avg_loss}")

def testing_loop(testing_dataloader_letters, loss_fn):
    total_loss = 0
    accuracy.reset()
    size = len(testing_dataloader_letters)
    with torch.no_grad():
        for images,labels in testing_dataloader_letters:
            output = net(images)
            loss = loss_fn(output, labels)
            total_loss += loss
            accuracy.update(output,labels)
    avg_loss = total_loss / size
    print(f"Avg Testing Accuracy: {accuracy.compute() * 100 :.2f}%")
    print(f"Avg Testing Loss: {avg_loss}")


In [84]:
for i in range(10):
    print(f"Epoch: {i}")
    testing_loop(testing_dataloader_letters, loss_fn)
    training_loop(training_dataloader_letters, optimizer, loss_fn)
    print("------------------")

Epoch: 0
Avg Testing Accuracy: 5.79%
Avg Testing Loss: 3.259567975997925
Avg Training Accuracy: 74.01%
Avg Training Loss: 0.8775403668880463
------------------
Epoch: 1
Avg Testing Accuracy: 85.32%
Avg Testing Loss: 0.47863802313804626
Avg Training Accuracy: 86.66%
Avg Training Loss: 0.4254660797119141
------------------
Epoch: 2
Avg Testing Accuracy: 87.91%
Avg Testing Loss: 0.3924606442451477
Avg Training Accuracy: 88.37%
Avg Training Loss: 0.3692239806652069
------------------
Epoch: 3
Avg Testing Accuracy: 88.86%
Avg Testing Loss: 0.36765965819358826
Avg Training Accuracy: 89.08%
Avg Training Loss: 0.34268516063690185
------------------
Epoch: 4
Avg Testing Accuracy: 89.19%
Avg Testing Loss: 0.35119861364364624
Avg Training Accuracy: 89.63%
Avg Training Loss: 0.3267808245420456
------------------
Epoch: 5
Avg Testing Accuracy: 89.85%
Avg Testing Loss: 0.3450244963169098
Avg Training Accuracy: 90.11%
Avg Training Loss: 0.312550124168396
------------------
Epoch: 6
Avg Testing Accura